<a href="https://colab.research.google.com/github/serrve1/AlphaRisk/blob/main/%EC%A0%84%EC%B2%98%EB%A6%AC(3%EB%B2%88).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# 코랩에서 한글깨짐을 방지
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf


plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] =False

pd.set_option('display.float_format', lambda x: '%.2f' % x)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [ ]:
#나눔고딕 깨짐문제 해결

%matplotlib inline

import matplotlib as mpl  # 기본 설정
import matplotlib.font_manager as fm  # 폰트 관리


!apt-get update -qq         # apt-get 패키지 설치 명령어, -qq : 에러외 메세지 숨기기
!apt-get install fonts-nanum* -qq #나눔글꼴 설치

fe = fm.FontEntry(fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', name='NanumGothic') #파일 저장되어있는 경로와 이름 설정
fm.fontManager.ttflist.insert(0, fe)  # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumGothic'}) #폰트설정

Selecting previously unselected package fonts-nanum-coding.
(Reading database ... 121772 files and directories currently installed.)
Preparing to unpack .../fonts-nanum-coding_2.5-3_all.deb ...
Unpacking fonts-nanum-coding (2.5-3) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-7_all.deb ...
Unpacking fonts-nanum-eco (1.000-7) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20200506-1_all.deb ...
Unpacking fonts-nanum-extra (20200506-1) ...
Setting up fonts-nanum-extra (20200506-1) ...
Setting up fonts-nanum-coding (2.5-3) ...
Setting up fonts-nanum-eco (1.000-7) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [ ]:
# 성장률 파일 불러오기
# grow = pd.read_csv('/content/gdrive/MyDrive/프로젝트2/데이터수집/성장성지표계산결과_1.csv', encoding= 'cp949', index_col=0)
grow = pd.read_csv('/content/gdrive/MyDrive/프로젝트2/데이터수집/성장성지표계산결과_2.csv', encoding= 'cp949', index_col=0)

grow.rename(columns={'영업이익증가율(IFRS)': '정상영업이익증가율(IFRS)'}, inplace=True)
grow.rename(columns={'영업이익증가율(IFRS연결)': '정상영업이익증가율(IFRS연결)'}, inplace=True)
grow = grow[['회사명', '거래소코드', '회계년도','매출액증가율(IFRS)',
       '정상영업이익증가율(IFRS)', '순이익증가율(IFRS)', '총자본증가율(IFRS)', '유동자산증가율(IFRS)',
       '비유동자산증가율(IFRS)', '유형자산증가율(IFRS)', '매출액증가율(IFRS연결)',
       '정상영업이익증가율(IFRS연결)', '순이익증가율(IFRS연결)', '총자본증가율(IFRS연결)',
       '유동자산증가율(IFRS연결)', '비유동자산증가율(IFRS연결)', '유형자산증가율(IFRS연결)']]

In [ ]:
# 파일 불러오기
file_df = pd.read_csv('/content/gdrive/MyDrive/프로젝트2/데이터수집/모델링_후보변수.csv', encoding='cp949')

file_df.rename(columns={'감사의견코드(IFRS)': '감사의견코드(IFRS)', '감사의견코드(IFRS).1': '감사의견코드(IFRS연결)'}, inplace=True)

In [ ]:
# 성장성 지표는 직접 산식을 대입하여 구하였기 때문에 ts에서 받아온 성장성 지표는 drop 처리
file_df.drop(['매출액증가율(IFRS)',
       '정상영업이익증가율(IFRS)', '순이익증가율(IFRS)', '총자본증가율(IFRS)', '유동자산증가율(IFRS)',
       '비유동자산증가율(IFRS)', '유형자산증가율(IFRS)',
 '[제조]매출액증가율(IFRS연결)',
 '[제조]총자본증가율(IFRS연결)',
 '[제조]정상영업이익증가율(IFRS연결)',
 '[제조]순이익증가율(IFRS연결)',
 '[제조]유동자산증가율(IFRS연결)',
 '[제조]비유동자산증가율(IFRS연결)',
 '[제조]유형자산증가율(IFRS연결)',], axis= 1, inplace= True)

In [ ]:
file_df[file_df['거래소코드'] == 65270]

,회사명,거래소코드,회계년도,배당성향(IFRS),자기자본순이익률(IFRS),유동비율(IFRS),당좌비율(IFRS),부채비율(IFRS),차입금의존도(IFRS),유동부채비율(IFRS),...,[제조]경영자본회전률(IFRS연결),[제조]유동자산회전률(IFRS연결),[제조]자기자본회전률(IFRS연결),[제조]재고자산회전기간(IFRS연결),[제조]매출채권회전기간(IFRS연결),[제조]매입채무회전기간(IFRS연결),합계_1인년간평균급여,합계_평균근속연수,감사의견코드(IFRS),감사의견코드(IFRS연결)
13254,(주)플렉스컴,65270,2011/12,0.00,12.52,148.78,105.63,123.85,29.77,117.08,...,1.43,2.20,3.23,71.69,40.56,41.31,31554.00,2.7,UQ,UQ
13255,(주)플렉스컴,65270,2012/12,0.00,26.24,167.55,115.99,96.24,11.20,89.16,...,2.08,2.87,4.73,56.56,28.76,38.69,32598.00,2.2년,UQ,UQ
13256,(주)플렉스컴,65270,2013/12,0.00,15.14,153.05,119.19,97.06,29.95,89.96,...,2.30,3.19,5.34,44.48,20.51,30.32,32487.00,2년5개월,UQ,UQ
13257,(주)플렉스컴,65270,2014/12,0.00,-13.62,98.60,61.66,94.41,30.48,89.82,...,1.01,1.65,3.25,100.76,29.96,43.64,53543.00,3년5개월,UQ,UQ
13258,(주)플렉스컴,65270,2015/12,0.00,0.00,29.64,26.54,0.00,56.42,0.00,...,1.20,2.31,11.89,99.55,15.34,54.78,54213.00,4년3개월,DU,DU


In [ ]:
# grow 파일과 file_df파일을 통합
grow_reset = grow.set_index('회계년도').reset_index()
file_df_reset = file_df.set_index('거래소코드').reset_index()

a = pd.concat([grow_reset, file_df_reset], axis=1, join='outer')
file_df = a.iloc[:, 3:]

# 회사명, 거래소 코드, 회계년도 열을 맨 앞으로 옮기기
file_df = file_df[['회사명', '거래소코드', '회계년도'] + [col for col in file_df.columns if col not in ['회사명', '거래소코드', '회계년도']]]

In [ ]:
# 회사명이 중복되는지 확인하기
print(f'파일에서 회사명을 기준으로 했을 때 기업 수 : {len(file_df["회사명"].unique())}')
print(f'파일에서 거래소코드를 기준으로 했을 때 기업 수 : {len(file_df["거래소코드"].unique())}')

파일에서 회사명을 기준으로 했을 때 기업 수 : 2850
파일에서 거래소코드를 기준으로 했을 때 기업 수 : 2878


In [ ]:
# 금융업은 제외
금융업 = pd.read_csv('/content/gdrive/MyDrive/프로젝트2/데이터수집/금융업_1.csv', encoding='cp949')
f_list = 금융업['거래소코드'].unique().tolist()
file_df = file_df[~file_df['거래소코드'].isin(f_list)]
file_df.reset_index(drop=True, inplace=True)

#### 전처리
+ 전처리 과정
  + IFRS 연결이나 개별 변수들이 모두 NAN 행은 삭제
  + 행 기준 연결 + 개별 -> 연결 | 연결 X + 개별 -> 개별 | 연결 X + 개별x -> drop


In [ ]:
print(f'원본 데이터 총 행의 수 {len(file_df)}')
print(f"원본 데이터 기업 수 {len(file_df['회사명'].unique())}")
print(f'컬럼 수 : {len(file_df.columns)}')

원본 데이터 총 행의 수 23434
원본 데이터 기업 수 2632
컬럼 수 : 129


In [ ]:
file_df.isnull().sum()

회사명                        0
거래소코드                      0
회계년도                       0
매출액증가율(IFRS)             597
정상영업이익증가율(IFRS)          584
                        ... 
[제조]매입채무회전기간(IFRS연결)    5396
합계_1인년간평균급여              253
합계_평균근속연수                419
감사의견코드(IFRS)             457
감사의견코드(IFRS연결)          5340
Length: 129, dtype: int64

In [ ]:
# IFRS(개별), IFRS연결을 분리하여 열의 갯수가 일치하는지 확인하기

file_ifrs_sep = file_df.filter(regex='\(IFRS\)')
file_ifrs_link = file_df.filter(regex='\(IFRS연결\)')

print(f'IFRS(개별) 컬럼의 수 : {len(file_ifrs_sep.columns)}')
print(f'IFRS(연결) 컬럼의 수 : {len(file_ifrs_link.columns)}')
print(f'컬럼 차이 : {len(file_ifrs_sep.columns) - len(file_ifrs_link.columns)}')

IFRS(개별) 컬럼의 수 : 65
IFRS(연결) 컬럼의 수 : 57
컬럼 차이 : 8


+ ifrs와 ifrs연결의 컬럼 수가 차이가 나는 이유 : 개별은 per(2), pbr(2), pcr(2), psr(2))가 있고 연결은 없음 --> 8개 차이가 남


In [ ]:
print(f'----------------개별 Nan 값----------------\n{file_ifrs_sep.isnull().sum()}')
print(f'----------------연결 Nan 값----------------\n{file_ifrs_link.isnull().sum()}')

----------------개별 Nan 값----------------
매출액증가율(IFRS)              597
정상영업이익증가율(IFRS)           584
순이익증가율(IFRS)              584
총자본증가율(IFRS)              584
유동자산증가율(IFRS)             584
                         ... 
PCR(최저)(IFRS)             758
PSR(최고)(IFRS)             758
PSR(최저)(IFRS)             758
[제조]이자보상배율(이자비용)(IFRS)    758
감사의견코드(IFRS)              457
Length: 65, dtype: int64
----------------연결 Nan 값----------------
매출액증가율(IFRS연결)                         5074
정상영업이익증가율(IFRS연결)                      5071
순이익증가율(IFRS연결)                         5071
총자본증가율(IFRS연결)                         5071
유동자산증가율(IFRS연결)                        5071
비유동자산증가율(IFRS연결)                       5071
유형자산증가율(IFRS연결)                        5071
[제조]자기자본순이익률(IFRS연결)                   5396
[제조]배당성향(IFRS연결)                      23434
[제조]매출액순이익률(IFRS연결)                    5396
[제조]총자본순이익률(IFRS연결)                    5396
[제조]경영자본순이익률(IFRS연결)                   5396
[제조]총자본사업이익률(IFRS연결)               

In [ ]:
print(f'----------------개별 Nan 값 (내림차순)----------------\n{file_ifrs_sep.isnull().sum().sort_values(ascending=False)}')
print(f'----------------연결 Nan 값 (내림차순)----------------\n{file_ifrs_link.isnull().sum().sort_values(ascending=False)}')

----------------개별 Nan 값 (내림차순)----------------
자본분배율(IFRS)       760
이윤분배율(IFRS)       760
부가가치율(IFRS)       760
설비투자효율(IFRS)      760
총자본투자효율(IFRS)     760
                 ... 
비유동자산증가율(IFRS)    584
유동자산증가율(IFRS)     584
총자본증가율(IFRS)      584
순이익증가율(IFRS)      584
감사의견코드(IFRS)      457
Length: 65, dtype: int64
----------------연결 Nan 값 (내림차순)----------------
[제조]배당성향(IFRS연결)                      23434
[제조]자본분배율(IFRS연결)                      5396
[제조]재고자산회전률(IFRS연결)                    5396
[제조]부가가치(IFRS연결)                       5396
[제조]총자본투자효율(IFRS연결)                    5396
[제조]설비투자효율(IFRS연결)                     5396
[제조]부가가치율(IFRS연결)                      5396
[제조]이윤분배율(IFRS연결)                      5396
[제조]총자본회전률(IFRS연결)                     5396
[제조]매입채무회전률(IFRS연결)                    5396
[제조]매출채권회전률(IFRS연결)                    5396
[제조]이자보상배율(이자비용)(IFRS연결)               5396
[제조]비유동장기적합률(IFRS연결)                   5396
[제조]유동비율(IFRS연결)                       5396
[제조]당좌비율(IFRS연결)     

In [ ]:
# ifrs개별와 ifrs연결 모든 컬럼에서 Nan인 값을 확인하고 제거
commom_col = ['회사명','거래소코드','회계년도']
sep_cop = file_ifrs_sep.columns.tolist()
link_sep = file_ifrs_link.columns.tolist()

sep_cop.remove('감사의견코드(IFRS)')
link_sep.remove('감사의견코드(IFRS연결)')


# per. pbr, psr, pcr 컬럼은 개별을 사용할지에 대해서 논의해야하기 때문에 따로 빼놓기
market_fe = ['PER(최고)(IFRS)', 'PER(최저)(IFRS)', 'PBR(최고)(IFRS)',
       'PBR(최저)(IFRS)', 'PCR(최고)(IFRS)', 'PCR(최저)(IFRS)', 'PSR(최고)(IFRS)',
       'PSR(최저)(IFRS)']

sep_cop = [x for x in sep_cop if x not in market_fe]

com_sep_link = sep_cop + link_sep

fake_df = file_df[com_sep_link]
all_nan_index = fake_df[fake_df.isnull().all(axis=1)].index.tolist()
all_nan = file_df.iloc[all_nan_index]

print(f'IFRS(개별), IFRS(연결) 값이 모두 없는 기업의 수 : {len(all_nan["회사명"].unique())}')
print(f'IFRS(개별), IFRS(연결) 값이 모두 없는 데이터 수 (행 기준) : {len(all_nan)}')

IFRS(개별), IFRS(연결) 값이 모두 없는 기업의 수 : 145
IFRS(개별), IFRS(연결) 값이 모두 없는 데이터 수 (행 기준) : 293


In [ ]:
# IFRS(개별), IFRS(연결) 모두가 없는 기업의 이름 확인하기
all_nan["회사명"].unique()

array(['(주)고려시멘트', '(주)골프존데카', '(주)구스앤홈', '(주)국전약품', '(주)금오하이텍', '(주)나이코',
       '(주)나인테크', '(주)네온테크', '(주)넥슨게임즈', '(주)누보', '(주)다린', '(주)다보링크',
       '(주)다이오진', '(주)대우증권그린코리아기업인수목적회사', '(주)더블유에스아이', '(주)데이드림엔터테인먼트',
       '(주)덴티스', '(주)드림시큐리티', '(주)디딤이앤에프', '(주)디와이디', '(주)라온테크', '(주)라이콤',
       '(주)라이프사이언스테크놀로지', '(주)러셀', '(주)레이크머티리얼즈', '(주)로보로보', '(주)모코엠시스',
       '(주)미래생명자원', '(주)미애부', '(주)바이오시네틱스', '(주)밸로프', '(주)본느',
       '(주)비앤에스미디어', '(주)비엔에프코퍼레이션', '(주)비투엔', '(주)서울상호저축은행', '(주)세기리텍',
       '(주)세림비앤지', '(주)셀바스헬스케어', '(주)스페이스솔루션', '(주)시냅스엠', '(주)신민상호저축은행',
       '(주)신세계아이앤씨', '(주)썸에이지', '(주)씨티네트웍스', '(주)아모그린텍', '(주)알로이스',
       '(주)앙츠', '(주)애니플러스', '(주)에스와이이노베이션', '(주)에이치앤아이', '(주)에치에프알',
       '(주)에코프로비엠', '(주)엔투텍', '(주)엔피', '(주)엘피케이로보틱스', '(주)엠앤씨생명과학',
       '(주)영현무역', '(주)예선테크', '(주)오션스톤', '(주)오하임앤컴퍼니', '(주)옵투스제약',
       '(주)와이즈버즈', '(주)우정바이오', '(주)원바이오젠', '(주)웨이버스', '(주)위메이드플레이',
       '(주)윙스풋', '(주)유디피', '(주)유쎌', '(주)이노인스트루먼트', '(주)이랜시스', '(주)이십일스토어',
       '(주)

In [ ]:
filter_df_1 = file_df.copy()

# all_nan_index를 제외한 행 선택
filter_df_1.drop(all_nan_index, inplace=True)
filter_df_1 = filter_df_1.reset_index(drop=True)
filter_df_1.head()

print(f'IFRS 값이 모두 없는 행을 제외한 행의 수 {len(filter_df_1)}')
print(f'IFRS 값이 모두 없는 기업을 제외한 기업의 수 {len(filter_df_1["회사명"].unique())}')

IFRS 값이 모두 없는 행을 제외한 행의 수 23141
IFRS 값이 모두 없는 기업을 제외한 기업의 수 2632


In [ ]:
filter_df_1.isnull().sum().sort_values(ascending=False).head(10)

[제조]배당성향(IFRS연결)                23141
상장폐지일                           21320
[제조]매출액순이익률(IFRS연결)              5103
[제조]CASH FLOW 대 부채비율(IFRS연결)     5103
[제조]매출액정상영업이익률(IFRS연결)           5103
[제조]총자본정상영업이익률(IFRS연결)           5103
[제조]총자본사업이익률(IFRS연결)             5103
[제조]경영자본순이익률(IFRS연결)             5103
[제조]총자본순이익률(IFRS연결)              5103
[제조]매입채무회전률(IFRS연결)              5103
dtype: int64

+ 기업들의 IFRS 처리 방식
  + 연결 -> 연결
  + 연결 + 개별 -> 연결
  + 개별 -> 개별

In [ ]:
link_col = file_ifrs_link.columns.tolist()
sep_col = file_ifrs_sep.columns.tolist()

# per. pbr, psr, pcr 컬럼은 개별을 사용할지에 대해서 논의해야하기 때문에 따로 빼놓기
market_fe = ['PER(최고)(IFRS)', 'PER(최저)(IFRS)', 'PBR(최고)(IFRS)',
       'PBR(최저)(IFRS)', 'PCR(최고)(IFRS)', 'PCR(최저)(IFRS)', 'PSR(최고)(IFRS)',
       'PSR(최저)(IFRS)']

sep_col = [x for x in sep_col if x not in market_fe]

# markert_fe를 빼면 개별과 연결은 컬럼의 수가 동일해야함
print(f'개별의 컬럼 수 : {len(sep_col)}')
print(f'연결의 컬럼 수 : {len(link_col)}')

개별의 컬럼 수 : 57
연결의 컬럼 수 : 57


In [ ]:
filter_df_1[link_col].isnull().sum()

매출액증가율(IFRS연결)                         4781
정상영업이익증가율(IFRS연결)                      4778
순이익증가율(IFRS연결)                         4778
총자본증가율(IFRS연결)                         4778
유동자산증가율(IFRS연결)                        4778
비유동자산증가율(IFRS연결)                       4778
유형자산증가율(IFRS연결)                        4778
[제조]자기자본순이익률(IFRS연결)                   5103
[제조]배당성향(IFRS연결)                      23141
[제조]매출액순이익률(IFRS연결)                    5103
[제조]총자본순이익률(IFRS연결)                    5103
[제조]경영자본순이익률(IFRS연결)                   5103
[제조]총자본사업이익률(IFRS연결)                   5103
[제조]총자본정상영업이익률(IFRS연결)                 5103
[제조]매출액정상영업이익률(IFRS연결)                 5103
[제조]CASH FLOW 대 부채비율(IFRS연결)           5103
[제조]CASH FLOW 대 차입금비율(IFRS연결)          5103
[제조]CASH FLOW 대 총자본비율(IFRS연결)          5103
[제조]CASH FLOW 대 매출액비율(IFRS연결)          5103
[제조]영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)     5045
[제조]   금융비용(IFRS연결)                    5045
[제조]      단기차입금(*)(IFRS연결)             5045
[제조]      유동성장기차입금의 감소(IFRS연결)  

In [ ]:
columns = filter_df_1.columns.tolist()

reset_sep_col = [
 '매출액증가율(IFRS)',
 '총자본증가율(IFRS)',
 '정상영업이익증가율(IFRS)',
 '순이익증가율(IFRS)',
 '유동자산증가율(IFRS)',
 '비유동자산증가율(IFRS)',
 '유형자산증가율(IFRS)',
 '자기자본순이익률(IFRS)',
 '유동비율(IFRS)',
 '당좌비율(IFRS)',
 '부채비율(IFRS)',
 '차입금의존도(IFRS)',
 '유동부채비율(IFRS)',
 '순운전자본비율(IFRS)',
 '현금비율(IFRS)',
 '유동자산구성비율(IFRS)',
 '자기자본구성비율(IFRS)',
 '비유동장기적합률(IFRS)',
 '매출액순이익률(IFRS)',
 '매출액정상영업이익률(IFRS)',
 '총자본순이익률(IFRS)',
 '경영자본순이익률(IFRS)',
 '총자본사업이익률(IFRS)',
 '총자본정상영업이익률(IFRS)',
 '재고자산회전률(IFRS)',
 '매출채권회전률(IFRS)',
 '매입채무회전률(IFRS)',
 '총자본회전률(IFRS)',
 '비유동자산회전률(IFRS)',
 '당좌자산회전률(IFRS)',
 '유형자산회전율(IFRS)',
 '경영자본회전률(IFRS)',
 '유동자산회전률(IFRS)',
 '자기자본회전률(IFRS)',
 '재고자산회전기간(IFRS)',
 '매입채무회전기간(IFRS)',
 '매출채권회전기간(IFRS)',
 'CASH FLOW 대 부채비율(IFRS)',
 'CASH FLOW 대 차입금비율(IFRS)',
 'CASH FLOW 대 총자본비율(IFRS)',
 'CASH FLOW 대 매출액비율(IFRS)',
 '영업활동으로 인한 현금흐름(간접법)(*)(IFRS)(천원)',
 '금융비용(IFRS)(천원)',
 '단기차입금(*)(IFRS)(천원)',
 '유동성장기차입금의 감소(IFRS)(천원)',
 '부가가치(IFRS)(백만원)',
 '총자본투자효율(IFRS)',
 '설비투자효율(IFRS)',
 '부가가치율(IFRS)',
 '이윤분배율(IFRS)',
 '자본분배율(IFRS)',
 '자기주식(IFRS)(천원)',
 '유동부채(*)(IFRS)(천원)',
 '유동자산(*)(IFRS)(천원)',
 '감사의견코드(IFRS)',
 '[제조]이자보상배율(이자비용)(IFRS)',
]
reset_link_col = [
 '매출액증가율(IFRS연결)',
 '총자본증가율(IFRS연결)',
 '정상영업이익증가율(IFRS연결)',
 '순이익증가율(IFRS연결)',
 '유동자산증가율(IFRS연결)',
 '비유동자산증가율(IFRS연결)',
 '유형자산증가율(IFRS연결)',
 '[제조]자기자본순이익률(IFRS연결)',
 '[제조]유동비율(IFRS연결)',
 '[제조]당좌비율(IFRS연결)',
 '[제조]부채비율(IFRS연결)',
 '[제조]차입금의존도(IFRS연결)',
 '[제조]유동부채비율(IFRS연결)',
 '[제조]순운전자본비율(IFRS연결)',
 '[제조]현금비율(IFRS연결)',
 '[제조]유동자산구성비율(IFRS연결)',
 '[제조]자기자본구성비율(IFRS연결)',
 '[제조]비유동장기적합률(IFRS연결)',
 '[제조]매출액순이익률(IFRS연결)',
 '[제조]매출액정상영업이익률(IFRS연결)',
 '[제조]총자본순이익률(IFRS연결)',
 '[제조]경영자본순이익률(IFRS연결)',
 '[제조]총자본사업이익률(IFRS연결)',
 '[제조]총자본정상영업이익률(IFRS연결)',
 '[제조]재고자산회전률(IFRS연결)',
 '[제조]매출채권회전률(IFRS연결)',
 '[제조]매입채무회전률(IFRS연결)',
 '[제조]총자본회전률(IFRS연결)',
 '[제조]비유동자산회전률(IFRS연결)',
 '[제조]당좌자산회전률(IFRS연결)',
 '[제조]유형자산회전율(IFRS연결)',
 '[제조]경영자본회전률(IFRS연결)',
 '[제조]유동자산회전률(IFRS연결)',
 '[제조]자기자본회전률(IFRS연결)',
 '[제조]재고자산회전기간(IFRS연결)',
 '[제조]매입채무회전기간(IFRS연결)',
 '[제조]매출채권회전기간(IFRS연결)',
 '[제조]CASH FLOW 대 부채비율(IFRS연결)',
 '[제조]CASH FLOW 대 차입금비율(IFRS연결)',
 '[제조]CASH FLOW 대 총자본비율(IFRS연결)',
 '[제조]CASH FLOW 대 매출액비율(IFRS연결)',
 '[제조]영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)',
 '[제조]   금융비용(IFRS연결)',
 '[제조]      단기차입금(*)(IFRS연결)',
 '[제조]      유동성장기차입금의 감소(IFRS연결)',
 '[제조]부가가치(IFRS연결)',
 '[제조]총자본투자효율(IFRS연결)',
 '[제조]설비투자효율(IFRS연결)',
 '[제조]부가가치율(IFRS연결)',
 '[제조]이윤분배율(IFRS연결)',
 '[제조]자본분배율(IFRS연결)',
 '[제조]         자기주식(IFRS연결)',
 '[제조]   유동부채(*)(IFRS연결)',
 '[제조]   유동자산(*)(IFRS연결)',
 '감사의견코드(IFRS연결)',
 '[제조]이자보상배율(이자비용)(IFRS연결)',
#  '배당성향(IFRS)',
#  'PER(최고)(IFRS)',
#  'PER(최저)(IFRS)',
#  'PBR(최고)(IFRS)',
#  'PBR(최저)(IFRS)',
#  'PCR(최고)(IFRS)',
#  'PCR(최저)(IFRS)',
#  'PSR(최고)(IFRS)',
#  'PSR(최저)(IFRS)',
#  '상장일',
#  '상장폐지일',
#  '합계_1인년간평균급여',
#  '합계_평균근속연수',
]

# filter_df_1 = filter_df_1.reindex(columns=new_order)

In [ ]:
print(len(reset_link_col))
print(len(reset_sep_col))

56
56


In [ ]:
import re

drop_market_fe = ['PER(최고)(IFRS)',	'PER(최저)(IFRS)',	'PBR(최고)(IFRS)',	'PBR(최저)(IFRS)',	'PCR(최고)(IFRS)',	'PCR(최저)(IFRS)',	'PSR(최고)(IFRS)',	'PSR(최저)(IFRS)', '[제조]배당성향(IFRS연결)']
save_cop_idx = filter_df_1[filter_df_1[reset_link_col].isnull().all(axis=1)].index.tolist()
# print(f'ifrs 연결이 없는 행의 수 {len(save_cop_idx)}')

final_df =  filter_df_1[~filter_df_1[reset_link_col].isnull().all(axis=1)]
final_df['회계기준'] = 'IFRS연결'

final_df.drop(reset_sep_col, axis=1, inplace=True)
final_df.drop(drop_market_fe, axis=1, inplace=True)

final_df.rename(columns=lambda x: re.sub(r'\(IFRS연결\)', '', x), inplace=True)
final_df.rename(columns=lambda x: re.sub(r'\(IFRS\)', '', x), inplace=True)

con1 = filter_df_1.iloc[save_cop_idx].copy()
con1['회계기준'] = 'IFRS개별'

con1.drop(reset_link_col, axis=1, inplace=True)
con1.drop(drop_market_fe, axis=1, inplace=True)

rename_dict = dict(zip(reset_sep_col, reset_link_col))
con1.rename(columns=rename_dict, inplace=True)

con1.rename(columns=lambda x: re.sub(r'\(IFRS연결\)', '', x), inplace=True)
con1.rename(columns=lambda x: re.sub(r'\(IFRS\)', '', x), inplace=True)

# print(con1)
df2 = pd.concat([final_df, con1], axis=0).reset_index(drop=True)

print(f'IFRS 연결을 쓰는 행의 수 {len(final_df)}')
print(f'IFRS 개별을 쓰는 행의 수 {len(con1)}')
print(f'총 데이터 행의 크기 {len(df2)}')

IFRS 연결을 쓰는 행의 수 19091
IFRS 개별을 쓰는 행의 수 4050
총 데이터 행의 크기 23141


In [ ]:
con1

,회사명,거래소코드,회계년도,매출액증가율,정상영업이익증가율,순이익증가율,총자본증가율,유동자산증가율,비유동자산증가율,유형자산증가율,...,[제조] 자기주식,[제조] 유동부채(*),[제조] 유동자산(*),[제조]이자보상배율(이자비용),상장일,상장폐지일,합계_1인년간평균급여,합계_평균근속연수,감사의견코드,회계기준
9,(주)CMG제약,58820,2020/12,9.96,-30.01,-92.95,1.62,-3.06,18.52,-3.06,...,0.00,13648964.00,145981120.00,79.74,2001/08/31,NaN,49342.00,3.9,UQ,IFRS개별
10,(주)CMG제약,58820,2021/12,7.78,-8.88,1372.35,1.95,-14.04,49.25,-14.04,...,0.00,13728982.00,125482013.00,80.23,2001/08/31,NaN,49209.00,4.2,UQ,IFRS개별
11,(주)CMG제약,58820,2022/12,18.31,79.55,-112.26,7.07,7.64,6.10,7.64,...,0.00,21902549.00,135069969.00,195.16,2001/08/31,NaN,55602.00,4.1,UQ,IFRS개별
85,(주)SNT에너지,100840,2012/12,18.12,329.38,318.68,5.89,11.00,0.22,11.00,...,0.00,88443397.00,161463967.00,77.44,2008/02/22,NaN,50747.00,7.8,UQ,IFRS개별
86,(주)SNT에너지,100840,2013/12,-12.98,11.05,78.15,6.45,12.29,-0.74,12.29,...,0.00,91989174.00,181309369.00,50.81,2008/02/22,NaN,54656.00,7.7,UQ,IFRS개별
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23124,흥구석유(주),24060,2018/12,6.87,45.37,14.78,-5.89,-21.19,-2.09,-21.19,...,-682108.00,3801903.00,12514903.00,12.66,1994/12/07,NaN,31195.00,6.02,UQ,IFRS개별
23125,흥구석유(주),24060,2019/12,-9.35,-54.16,249.94,11.24,80.74,-2.63,80.74,...,-682108.00,4660149.00,22619714.00,252.02,1994/12/07,NaN,30730.00,6.23,UQ,IFRS개별
23126,흥구석유(주),24060,2020/12,-23.78,-198.61,-42.22,0.07,22.41,-8.21,22.41,...,-682108.00,2498772.00,27688393.00,999999999.00,1994/12/07,NaN,32637.00,7.54,UQ,IFRS개별
23127,흥구석유(주),24060,2021/12,15.12,94.16,-70.81,-1.28,-4.24,0.18,-4.24,...,-682108.00,2902084.00,26514523.00,999999999.00,1994/12/07,NaN,31291.00,7.61,UQ,IFRS개별


#### 회계년도가 다른 (03월 기준 행 DROP 하기)

In [ ]:
df3 = df2[df2['회계년도'].str.contains('/12')]

In [ ]:
df3.isnull().sum().sort_values(ascending= False)

상장폐지일           20855
[제조]이윤분배율        1016
[제조]설비투자효율       1016
[제조]매입채무회전기간     1016
[제조]자본분배율        1016
                ...  
회사명                 0
상장일                 0
거래소코드               0
회계년도                0
회계기준                0
Length: 65, dtype: int64

In [ ]:
# 2011년도 성장성 지표는 ifrs가 2011년에 도입 됐기 때문에 해당 연도를 삭제
df3 = df2[~df2['회계년도'].str.contains('2011/')]

In [ ]:
label = pd.read_csv('/content/gdrive/MyDrive/프로젝트2/데이터수집/부실기업_label.csv')

In [ ]:
df3['년도'] = df3['회계년도'].apply(lambda x : int(x[:4]))
# df3.info()

In [ ]:
부실기업_len = []
df_label = df3.copy()
df_label['label'] = 0
for i in range(0, len(label)):
  y = label.iloc[i]['년도']
  code = label.iloc[i]['거래소코드']

  # y와 code가 df_label에 있는지 확인하여 label1 값을 부여
  if ((df3['년도'] == y) & (df3['거래소코드'] == code)).any():
      df_label.loc[(df_label['년도'] == y) & (df_label['거래소코드'] == code), 'label'] = 1
      부실기업_len.append(i)

In [ ]:
df_label['label'].value_counts()

0    20524
1      928
Name: label, dtype: int64

In [ ]:
df_label_1 = df_label[df_label['label'] == 1]
drop_cop_list = df_label[df_label['label'] == 1]['거래소코드'].tolist()
df_label_0 = df_label[~df_label['거래소코드'].isin(drop_cop_list)]
# print(len(df_label_0))
df_label2 = pd.concat([df_label_1, df_label_0], axis=0).reset_index(drop=True)

In [ ]:
len(df_label2)

14159

In [ ]:
df_label2['label'].value_counts()

0    13231
1      928
Name: label, dtype: int64

In [ ]:
df_label2.rename(columns=lambda x: re.sub(r'\[제조\]', '', x).strip(), inplace=True)

df_label2.columns

Index(['회사명', '거래소코드', '회계년도', '매출액증가율', '정상영업이익증가율', '순이익증가율', '총자본증가율',
       '유동자산증가율', '비유동자산증가율', '유형자산증가율', '배당성향', '자기자본순이익률', '매출액순이익률',
       '총자본순이익률', '경영자본순이익률', '총자본사업이익률', '총자본정상영업이익률', '매출액정상영업이익률',
       'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율', 'CASH FLOW 대 총자본비율',
       'CASH FLOW 대 매출액비율', '영업활동으로 인한 현금흐름(간접법)(*)', '금융비용', '단기차입금(*)',
       '유동성장기차입금의 감소', '부가가치', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '자기주식', '유동자산(*)', '유동부채(*)', '이자보상배율(이자비용)', '상장일', '상장폐지일', '유동비율',
       '당좌비율', '부채비율', '차입금의존도', '유동부채비율', '순운전자본비율', '현금비율', '유동자산구성비율',
       '자기자본구성비율', '비유동장기적합률', '재고자산회전률', '매출채권회전률', '매입채무회전률', '총자본회전률',
       '비유동자산회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '유동자산회전률', '자기자본회전률',
       '재고자산회전기간', '매출채권회전기간', '매입채무회전기간', '합계_1인년간평균급여', '합계_평균근속연수',
       '감사의견코드', '회계기준', '년도', 'label'],
      dtype='object')

In [ ]:
df_label2 = df_label2[~df_label2['설비투자효율'].isna()]

In [ ]:
df_label2.isnull().sum().sort_values(ascending=False).head(10)

상장폐지일        13088
매출액증가율         486
총자본증가율         482
유형자산증가율        482
유동자산증가율        482
비유동자산증가율       482
순이익증가율         482
정상영업이익증가율      482
합계_평균근속연수      153
배당성향           111
dtype: int64

In [ ]:
df_label2['합계_1인년간평균급여'].describe()

count    13650.00
mean     52756.75
std      21504.09
min          0.00
25%      40205.75
50%      49000.00
75%      61000.00
max     579000.00
Name: 합계_1인년간평균급여, dtype: float64

In [ ]:
df_label2[['합계_1인년간평균급여', '거래소코드', '년도']]

,합계_1인년간평균급여,거래소코드,년도
0,47000.00,990,2013
1,52667.00,97230,2013
2,47000.00,11810,2014
3,63000.00,2710,2014
4,54692.00,217730,2017
...,...,...,...
14154,31195.00,24060,2018
14155,30730.00,24060,2019
14156,32637.00,24060,2020
14157,31291.00,24060,2021


In [ ]:
len(df_label2[df_label2['매출액증가율'] == 0])

330

In [ ]:
df_label2.describe()

,거래소코드,매출액증가율,정상영업이익증가율,순이익증가율,총자본증가율,유동자산증가율,비유동자산증가율,유형자산증가율,배당성향,자기자본순이익률,...,유형자산회전율,경영자본회전률,유동자산회전률,자기자본회전률,재고자산회전기간,매출채권회전기간,매입채무회전기간,합계_1인년간평균급여,년도,label
count,13719.00,13233.00,13237.00,13237.00,13237.00,13237.00,13237.00,13237.00,13608.00,13719.00,...,13719.00,13719.00,13719.00,13719.00,13719.00,13719.00,13719.00,13650.00,13719.00,13719.00
mean,86243.26,inf,43.84,-203.11,9.23,10.19,13.33,10.19,35.73,-4.87,...,12.93,1.09,1.97,1.88,71.79,67.33,35.79,52756.75,2017.42,0.07
std,118624.11,NaN,888.51,20013.33,32.24,47.14,55.83,47.14,638.84,864.52,...,89.36,0.62,1.22,1.75,2178.71,155.80,441.76,21504.09,3.17,0.25
min,20.00,-99.69,-26620.87,-2233014.16,-93.41,-97.71,-96.96,-97.71,0.00,-101015.66,...,0.00,0.00,0.00,0.00,0.00,-4.74,0.00,0.00,2012.00,0.00
25%,14190.00,-4.02,-31.27,-43.20,-0.68,-4.99,-2.11,-4.99,0.00,1.41,...,1.82,0.70,1.19,0.87,18.07,38.09,14.91,40205.75,2015.00,0.00
50%,50960.00,4.85,2.77,2.31,4.89,5.09,3.76,5.09,12.16,6.11,...,3.06,0.98,1.76,1.42,41.29,56.25,25.53,49000.00,2018.00,0.00
75%,104200.00,16.39,50.56,71.09,13.30,17.53,15.09,17.53,29.49,11.64,...,5.75,1.34,2.47,2.30,69.66,79.12,39.94,61000.00,2020.00,0.00
max,950220.00,inf,53516.84,199855.25,1720.86,3110.47,2005.64,3110.47,68936.10,428.57,...,5058.87,10.13,17.77,36.17,254780.81,13774.30,50880.47,579000.00,2022.00,1.00


## 매출액 증가율이 inf 인 경우

- (주)파멥신 : 2019년 매출액이 0 > 0으로 대체
- (주)셀리드 : 2020년 매출액이 0 > 0으로 대체
- (주)카이노스메드 : 2020년 매출액이 0 > 0으로 대체

In [ ]:
df_label2[df_label2['매출액증가율'] == np.inf]

,회사명,거래소코드,회계년도,매출액증가율,정상영업이익증가율,순이익증가율,총자본증가율,유동자산증가율,비유동자산증가율,유형자산증가율,...,자기자본회전률,재고자산회전기간,매출채권회전기간,매입채무회전기간,합계_1인년간평균급여,합계_평균근속연수,감사의견코드,회계기준,년도,label
430,(주)파멥신,208340,2020/12,inf,-152.44,-202.00,-17.23,-26.32,31.92,-26.32,...,0.00,0.00,0.00,0.00,38798.00,3년3개월,UQ,IFRS연결,2020,1
770,(주)셀리드,299660,2021/12,inf,-146.11,-216.78,17.93,60.37,2.81,60.37,...,0.02,0.00,0.00,0.00,49374.00,1년4개월,UQ,IFRS개별,2021,1
5849,(주)카이노스메드,284620,2021/12,inf,-19.54,-41.74,20.64,15.27,82.11,15.27,...,0.12,0.00,0.00,0.00,73000.00,3년3월,UQ,IFRS연결,2021,0


In [ ]:
df_label2.loc[df_label2['매출액증가율'] == np.inf, '매출액증가율'] = 0

#### 이자보상배율, 상장폐지일 drop

In [ ]:
df_label2.drop(['이자보상배율(이자비용)', '상장폐지일'], axis = 1, inplace= True)

====================================================================================================================================================

#### 성장성 지표들의 결측치인 값은 더이상 구할 수 없다고 판단, drop 처리

In [ ]:
df_label2[df_label2['매출액증가율'].isna()]['회사명'].unique()

array(['(주)강스템바이오텍', '(주)광림', '(주)넥스지', '(주)드래곤플라이', '(주)디엔에이링크',
       '(주)라닉스', '(주)레고켐바이오사이언스', '(주)메드팩토', '(주)모린스', '(주)상상인인더스트리',
       '(주)세영디앤씨', '(주)스킨앤스킨', '(주)신테카바이오', '(주)씨아이테크', '(주)아이톡시',
       '(주)액션스퀘어', '(주)에스마크', '(주)에스메디', '(주)에이프로젠헬스케어앤게임즈', '(주)엔에스브이',
       '(주)엔젠바이오', '(주)엔지스테크널러지', '(주)엔터파트너즈', '(주)유수홀딩스', '(주)유진로봇',
       '(주)이글루코퍼레이션', '(주)인스코비', '(주)인터플렉스', '(주)제이웨이', '(주)티라유텍',
       '(주)티사이언티픽', '(주)퓨전', '(주)해피드림', '다이노나(주)', '디에스티(주)',
       '디엑스앤브이엑스(주)', '메디앙스(주)', '에이스하이텍(주)', '원텍(주)', '제이준코스메틱(주)',
       '지엘팜텍(주)', '포스코스틸리온(주)', '한솔인티큐브(주)', '(주)박셀바이오', '(주)큐리언트',
       '(주)SNT에너지', '(주)국전약품', '(주)금화피에스시', '(주)나스미디어', '(주)나이스디앤비',
       '(주)나인테크', '(주)네오셈', '(주)네오이뮨텍', '(주)네오티스', '(주)네오팜', '(주)녹십자셀',
       '(주)누보', '(주)다원시스', '(주)대덕', '(주)대림제지', '(주)대유', '(주)대유에이피',
       '(주)대한과학', '(주)더블유게임즈', '(주)더블유에스아이', '(주)덕산테코피아', '(주)덕신하우징',
       '(주)덴티스', '(주)도화엔지니어링', '(주)동구바이오제약', '(주)동성화인텍', '(주)동우팜투테이블',
       '(주)동원개발', '(주)드림시큐리티', '(주)디모아

#### 매출액 증가율 중에서 동일한 거래소코드 혹은 기업에서 결측치가 2번 이상 나타나는 문제
- 회사명 기준 두 번 이상 나타나는 행 : 88
- 회사명 기준 두 번 이상 나타나는 회사의 수 : 41
- 거래소코드 기준 두 번 이상 나타나는 행: 71
- 거래소 코드 기준 두 번 이상 나타나는 회사의 수 : 34

-  회사명은 같으나, 거래소 코드 변경으로 인해 생기는 문제 존재
-  매출액을 구할 수 없는 경우도 존재
    - 상장일과 회계년도 비교 결과, 신규상장으로 인한 문제는 아닌것으로 판단

In [ ]:
# 데이터프레임에서 '매출액증가율'이 결측값인 행을 필터링합니다
na_sales_growth = df_label2[df_label2['매출액증가율'].isna()]

# '회사명'을 기준으로 그룹화하여 두 번 이상 나타나는 회사를 찾습니다
duplicate_companies = na_sales_growth[na_sales_growth.duplicated(subset='거래소코드', keep=False)]
len(duplicate_companies['거래소코드'].unique())

34

#### 매출액 증가율 중복 컬럼 check
-  패션플랫폼(주) > 거래소 코드 변경
    - 2016년 256090 (당시 label 0 : 부실로인한 상장폐지는 아닌듯? )
    - 2017년 x
    - 2018년 225590

In [ ]:
df_label2[df_label2['거래소코드'] == 256090]

,회사명,거래소코드,회계년도,매출액증가율,정상영업이익증가율,순이익증가율,총자본증가율,유동자산증가율,비유동자산증가율,유형자산증가율,...,자기자본회전률,재고자산회전기간,매출채권회전기간,매입채무회전기간,합계_1인년간평균급여,합계_평균근속연수,감사의견코드,회계기준,년도,label
13957,패션플랫폼(주),256090,2016/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.04,100.46,40.98,39.92,53000.00,2년10개월,UQ,IFRS개별,2016,0


#### 전처리 전 증가율 값 NaN 값 드랍한 dataset : df_label3

In [ ]:
df_label3 = df_label2[~df_label2['매출액증가율'].isna()]

## 결측치

df_label3 에서 결측치
- 배당성향                      111
- 합계_1인년간평균급여                69
- 합계_평균근속연수                 149

#### 배당성향
- 111 rows
- 회계기준 모두 IFRS 연결
- 23개 회사(거래소코드 기준)

In [ ]:
len(df_label3[df_label3['배당성향'].isna()]['거래소코드'].unique())

23

In [ ]:
df_label3[df_label3['배당성향'].isna()]

,회사명,거래소코드,회계년도,매출액증가율,정상영업이익증가율,순이익증가율,총자본증가율,유동자산증가율,비유동자산증가율,유형자산증가율,...,자기자본회전률,재고자산회전기간,매출채권회전기간,매입채무회전기간,합계_1인년간평균급여,합계_평균근속연수,감사의견코드,회계기준,년도,label
531,로스웰인터내셔널유한회사,900260,2022/12,188.77,-167.89,-208.34,-1.52,-6.73,16.69,-6.73,...,0.42,77.87,100.92,103.29,0.00,2.49,UQ,IFRS연결,2022,1
616,엘브이엠씨홀딩스(주),900140,2020/12,-24.62,-77.46,55.83,-4.59,-7.20,-2.16,-7.20,...,0.80,163.97,119.53,41.84,103745.00,6년3개월,UQ,IFRS연결,2020,1
626,완리인터내셔널홀딩스,900180,2017/12,-21.26,-28.28,-13.72,-2.48,-9.67,1.58,-9.67,...,0.52,61.15,115.24,12.31,NaN,NaN,DS,IFRS연결,2017,1
634,웨이포트유한공사,900130,2014/12,10.78,-42.24,-1.19,11.97,7.82,21.52,7.82,...,6.52,115.33,87.19,171.57,NaN,NaN,UQ,IFRS연결,2014,1
669,중국고섬공고유한공사,950070,2012/12,-12.82,-87.69,58.41,20.12,-39.92,44.86,-39.92,...,0.41,31.10,20.37,87.99,NaN,NaN,DS,IFRS연결,2012,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11603,헝셩그룹유한회사,900270,2018/12,-0.15,-11.49,-22.61,7.93,10.99,-2.68,10.99,...,0.87,8.81,170.04,23.67,NaN,NaN,UQ,IFRS연결,2018,0
11604,헝셩그룹유한회사,900270,2019/12,10.38,27.13,31.19,11.93,16.46,-5.98,16.46,...,0.85,8.83,182.74,20.17,NaN,NaN,UQ,IFRS연결,2019,0
11605,헝셩그룹유한회사,900270,2020/12,-35.14,-88.61,-84.25,-0.29,-3.71,16.44,-3.71,...,0.51,12.06,229.96,31.98,NaN,NaN,UQ,IFRS연결,2020,0
11606,헝셩그룹유한회사,900270,2021/12,-15.09,-40.95,-88.10,12.09,15.25,-0.74,15.25,...,0.41,14.04,217.46,37.23,NaN,NaN,UQ,IFRS연결,2021,0


#### 합계_1인년간평균급여
- 69 rows
- 이 컬럼이 결측치인 컬럼은 모두 합계_평균근속연수도 결측치
- 24개 회사(거래소코드 기준)

In [ ]:
df_label3[df_label3['합계_1인년간평균급여'].isna()]

,회사명,거래소코드,회계년도,매출액증가율,정상영업이익증가율,순이익증가율,총자본증가율,유동자산증가율,비유동자산증가율,유형자산증가율,...,자기자본회전률,재고자산회전기간,매출채권회전기간,매입채무회전기간,합계_1인년간평균급여,합계_평균근속연수,감사의견코드,회계기준,년도,label
185,(주)씨유전자,56340,2013/12,-76.23,43.67,-47.47,-51.23,-79.55,-39.08,-79.55,...,0.37,30.71,155.41,186.98,NaN,NaN,UQ,IFRS연결,2013,1
586,애머릿지코퍼레이션,900100,2013/12,-24.82,-5409.42,-8214.58,-25.11,-22.40,-31.09,-22.40,...,10.23,57.98,46.78,32.70,NaN,NaN,UQ,IFRS연결,2013,1
626,완리인터내셔널홀딩스,900180,2017/12,-21.26,-28.28,-13.72,-2.48,-9.67,1.58,-9.67,...,0.52,61.15,115.24,12.31,NaN,NaN,DS,IFRS연결,2017,1
634,웨이포트유한공사,900130,2014/12,10.78,-42.24,-1.19,11.97,7.82,21.52,7.82,...,6.52,115.33,87.19,171.57,NaN,NaN,UQ,IFRS연결,2014,1
669,중국고섬공고유한공사,950070,2012/12,-12.82,-87.69,58.41,20.12,-39.92,44.86,-39.92,...,0.41,31.10,20.37,87.99,NaN,NaN,DS,IFRS연결,2012,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11606,헝셩그룹유한회사,900270,2021/12,-15.09,-40.95,-88.10,12.09,15.25,-0.74,15.25,...,0.41,14.04,217.46,37.23,NaN,NaN,UQ,IFRS연결,2021,0
11607,헝셩그룹유한회사,900270,2022/12,8.01,46.60,-420.05,-2.64,-1.63,-7.43,-1.63,...,0.42,12.45,170.96,31.28,NaN,NaN,UQ,IFRS연결,2022,0
12018,(주)드림티엔터테인먼트,220110,2019/12,-28.60,1999.46,103.02,-57.30,98.57,-86.34,98.57,...,3.85,0.00,11.32,0.00,NaN,NaN,DS,IFRS개별,2019,0
12947,(주)토마토시스템,393210,2022/12,18.52,399.45,255.97,50.10,123.13,-26.72,123.13,...,3.49,0.00,23.39,5.53,NaN,NaN,UQ,IFRS개별,2022,0


In [ ]:
len(df_label3[df_label3['합계_1인년간평균급여'].isna()]['거래소코드'].unique())

24

#### 합계_평균근속연수                 
- 149 rows
- 58개 회사(거래소코드 기준 )
- 이 컬럼이 결측치인 경우 위 결측치들 모두 0 혹은 NaN

In [ ]:
len(df_label3[df_label3['합계_평균근속연수'].isna()]['거래소코드'].unique())

58

In [ ]:
df_label3[df_label3['합계_평균근속연수'].isna()][['거래소코드', '배당성향', '합계_평균근속연수', '합계_1인년간평균급여', 'label']]

,거래소코드,배당성향,합계_평균근속연수,합계_1인년간평균급여,label
36,126870,0.00,NaN,37700.00,1
79,4790,0.00,NaN,38000.00,1
184,37340,0.00,NaN,40071.00,1
185,56340,0.00,NaN,NaN,1
214,179530,0.00,NaN,41000.00,1
...,...,...,...,...,...
13020,114630,19.85,NaN,32761.00,0
13021,114630,27.58,NaN,34062.00,0
13025,94940,20.42,NaN,42382.00,0
13415,1880,0.00,NaN,67929.00,0


- 모두 중복되어 결측치가 있었기 때문
- 0값이 더 많아서 drop 해도 괜찮을 것으로 판단
- 해당 row들 drop 처리

In [ ]:
df_label4 = df_label3[~df_label3['합계_평균근속연수'].isna()]
df_label4 = df_label4[~df_label4['배당성향'].isna()]
df_label4.isnull().sum().sort_values(ascending=False)

회사명        0
유동자산(*)    0
상장일        0
유동비율       0
당좌비율       0
          ..
총자본투자효율    0
설비투자효율     0
부가가치율      0
이윤분배율      0
label      0
Length: 65, dtype: int64